In [5]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch import nn

In [2]:
train_dataset = datasets.MNIST(
    root="data",
    train = True,
    download = True,
    transform = ToTensor()
)
test_dataset = datasets.MNIST(
    root="data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [18]:
hyper_parameters = {}
hyper_parameters['batch_size'] = 64
hyper_parameters['epochs'] = 20
hyper_parameters['lr'] = 1e-4
hyper_parameters['weight_decay'] = 1e-5
hyper_parameters['loss_fn'] = nn.CrossEntropyLoss()

In [15]:
train_loader = DataLoader(train_dataset, batch_size=hyper_parameters['batch_size'], shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=hyper_parameters['batch_size'], shuffle=False, num_workers=8)


In [12]:
def train_loop(dataloader, model, loss_fn, optimizer, show_step = True):
    model.train()
    ls = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ls = loss.item()
        # if batch % 100 == 0 and show_step:
        #     loss, current = loss.item(), batch *32+len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return ls

def test_loop(dataloader, model, loss_fn, show_step = True):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    if show_step:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct
def loop_epoch(epoch, train_loader, test_loader, model, loss_fn, optimizer, show_step):
    acc_history = []
    for t in range(epoch):
        if show_step:
            print(f"Epoch {t+1}\n-------------------------------")
        else:
            print(f"\r Epoch{t+1} running", end="")
        train_loop(train_loader, model, loss_fn, optimizer, show_step)
        acc = test_loop(test_loader, model, loss_fn, show_step)
        acc_history.append(acc)
    print("Done!")
    return acc_history

In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=1, stride=1, padding=1),  #28->(27/n)+1
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2), #(25/n)+1
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1, padding=3),#(23/n)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),
        )

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*15*15, out_features=28*15*15),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(num_features=28*15*15),
            nn.Linear(in_features=28*15*15, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=10),
        )

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat((x1, x2, x3), 1)
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x


model = ConvNet().to(device)
hyper_parameters['optimizer'] = torch.optim.Adam(model.parameters(), lr=hyper_parameters['lr'])

In [16]:
acc_history = loop_epoch(hyper_parameters['epochs'], train_loader, test_loader, model, hyper_parameters['loss_fn'], hyper_parameters['optimizer'], show_step=False)

 Epoch5 runningDone!


In [17]:
print(f"Accuracy history: {acc_history}")

Accuracy history: [0.956, 0.9748, 0.9814, 0.9831, 0.9829]
